# Virtual Realtor Play Ground (copy of Lab 3 - OpenAI Agents SDK with streaming)


In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, function_tool
from openai.types.responses import ResponseTextDeltaEvent
import gradio as gr
import os
import requests

load_dotenv(override=True)
name = "Monika Trivedi"
company_name = "MonikaRealty"



True

In [ ]:
instructions = """
You represent the AI Digital Twin of a human Realtor. That Realtor can be any licensed realtor named Monika Trivedi. 
You are friendly and amiable, and you introduce yourself as Your Virtal Realtor.
You let the user know that you represent Monika Trivedi of MonikaRealty and can answer questions on 
Monika's behalf. 
Monika Tricedi is a licenced realtor in the state of California. 
Monika loves what she does. She loves helping people find their dream home. 
She also helps people sell their homes. She specialized in upgarades. 
You chat with visitors on Monika's personal website. You answer questions about Kinjal's work any property 
they are inquiring about as Monika would. 
You will be provided with latest informatoion about current properties in the market via various means. 
If you don't know the answer, say so.

You are a helpful assistant that can answer questions about the properties in the market.
use the latest information provided to you to answer the questions.
If asked about a property, gather the address of the property and use the property_info_tool to answer the questions.
If you don't know the answer, gather the address of the propert, users contact information and send a push notification to Monika to tell her the question you couldn't answer, so that she can follow up 
and adds it to the knowledge base.
If you don't know the answer, say so.

"""

In [20]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [ ]:
def find_property_info(message: str):
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

@function_tool
def book_appointment(message: str, name: str, phone: str, email: str) -> str:
    """ Book a calender appointment with the user for a property visit or a meeting. uing google drive calendar. 

    Args:
        message: The address of the property to find the info for.
    """

    find_property_info(message)
    return "Push notification sent"


@function_tool
def property_info_tool(message: str) -> str:
    """ find the property info for the given address by calling an API. 

    Args:
        message: The address of the property to find the info for.
    """

    find_property_info(message)
    return "Push notification sent"

def send_push_notification(message: str):
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

@function_tool
def push(message: str) -> str:
    """Send a text message as a push notification to Kinjal with this brief message

    Args:
        message: The short text message to push to Kinjal.
    """

    send_push_notification(message)
    return "Push notification sent"

In [26]:
agent = Agent(name="VirtualRealtor", instructions=instructions, model="gpt-4.1-mini", tools=[push, property_info_tool])

In [27]:
async def chat(message, history):
    messages = [{"role": prior["role"], "content": prior["content"]} for prior in history]  
    messages += [{"role": "user", "content": message}]
    response = await Runner.run(agent, messages)
    return response.final_output

In [28]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [7]:
async def chat(message, history):
    messages = [{"role": prior["role"], "content": prior["content"]} for prior in history]  
    messages += [{"role": "user", "content": message}]
    response = Runner.run_streamed(agent, messages)
    reply = ""
    async for event in response.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            reply += event.data.delta
            yield reply

In [8]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
